In [103]:
import os

In [104]:
os.chdir('../')

In [105]:
import os
os.chdir('/Users/shrishtiroy/Documents/projects/text-summarizer')
print(os.getcwd())  # Should print your project root
print("contents", os.listdir(os.getcwd()))  # Should show 'config' in the list

/Users/shrishtiroy/Documents/projects/text-summarizer
contents ['research', 'artifacts', '.DS_Store', 'LICENSE', 'requirements.txt', 'config', 'Dockerfile', 'params.yaml', 'README.md', 'test_artifacts', 'setup.py', 'logs', '.gitignore', 'test_dir', '.github', 'app.py', 'template.py', '.git', 'main.py', 'src']


In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [ ]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [ ]:
#read the config.yaml and params.yaml files
#create directories based on the config.yaml file
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config.artifacts_root)])

    #get from config.yaml file    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([Path(config.root_dir)])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config
    
        

In [109]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [ ]:
import os, zipfile
#ingests all the information from the files initialized by ConfigurationManager
class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [113]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[level=logging.INFO, format='[2025-06-02 11:27:26,859]: INFO: common: YAML file: config/config.yaml loaded successfully.:]
[level=logging.INFO, format='[2025-06-02 11:27:26,860]: INFO: common: YAML file: params.yaml loaded successfully.:]
Creating directories...
[level=logging.INFO, format='[2025-06-02 11:27:26,861]: INFO: common: Creating directory: artifacts:]
Creating directories...
[level=logging.INFO, format='[2025-06-02 11:27:26,861]: INFO: common: Creating directory: artifacts/data_ingestion:]
[level=logging.INFO, format='[2025-06-02 11:27:26,862]: INFO: 2525449871: File already exists of size: 7718 KB:]
